# Convert our own Handwriting into compatible datatype

In [13]:
import numpy as np
import tensorflow as tf
from scipy import ndimage
import math
import cv2
import os
#For Debugging
np.set_printoptions(threshold=np.nan)
# create an array where we can store our 12 characters
images = np.zeros((12,784))
# and the labels
correct_vals = np.zeros((12,46))

# WE WANT ML 2018 using the labels specified in 'EMNIST Import and Application by SKLearn'
i = 0
labels = [14,32,21,24,31,32,22,21,2,0,1,8]

for letter in ["W","E1","L1","O","V","E2","M","L2","2","0","1","8"]:
    # read the image
    gray = cv2.imread(letter+"booth.png", 0)
    #invert the image, convert to grayscale using the threshold of 50 / 225 to decide whether or not it is actually a pencil mark
    _, gray = cv2.threshold(255-gray, 50, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    #gray = cv2.resize(gray,(28, 28))

    
    #Remove any lines of all black
    while np.sum(gray[0]) == 0:
        gray = gray[1:]

    while np.sum(gray[:,0]) == 0:
        gray = np.delete(gray,0,1)

    while np.sum(gray[-1]) == 0:
        gray = gray[:-1]

    while np.sum(gray[:,-1]) == 0:
        gray = np.delete(gray,-1,1)
    
    rows,cols = gray.shape
    
    #gray = np.lib.pad(gray,(rowsPadding,colsPadding),'constant') 
    #Calculate how uch buffer is needed to add to the image to make it square again
    if rows > cols:
        factor = 20.0/rows
        rows = 20
        cols = int(round(cols*factor))
        gray = cv2.resize(gray, (cols,rows))
    else:
        factor = 20.0/cols
        cols = 20
        rows = int(round(rows*factor))
        gray = cv2.resize(gray, (cols, rows))
    colsPadding = (int(math.ceil((28-cols)/2.0)),int(math.floor((28-cols)/2.0)))
    rowsPadding = (int(math.ceil((28-rows)/2.0)),int(math.floor((28-rows)/2.0)))
    #Center the image
    cy,cx = ndimage.measurements.center_of_mass(gray)
    rows,cols = gray.shape
    shiftx = np.round(cols/2.0-cx).astype(int)
    shifty = np.round(rows/2.0-cy).astype(int)
    #Add the padding to make the image square again
    gray = np.lib.pad(gray,(rowsPadding,colsPadding),'constant')
    gray = cv2.resize(gray,(28, 28))
    #The current images have a range of 0-255 and we want to convert it to between 0-1
    flatten = gray.flatten() / 255.0
    #Write the images
    cv2.imwrite(letter+"boothPRO.png", gray)
    #print (gray)
    #print (flatten)
    #print(len(images[i]))
    #print(len(flatten))
    #Keep track of the data for each image as well as the correct labels 
    images[i] = flatten
    correct_val = np.zeros((46))
    correct_val[labels[i]] = 1
    correct_vals[i] = correct_val
    i += 1

#print (images)
#Save the binary images
np.save("binaryLetters.npy",images)